In [15]:
brain_region_list = ['left_ALM', 'left_BLA', 'left_ECT', 'left_Medulla', 'left_Midbrain', 'left_Striatum', 'left_Thalamus'] +\
                    ['right_ALM', 'right_BLA', 'right_ECT', 'right_Medulla', 'right_Midbrain', 'right_Striatum', 'right_Thalamus']


In [4]:
import os
import re

directory_path = '../data/raw_data/neural_data_left_ALM'
entries = os.listdir(directory_path)

In [12]:
brain_region = 'left_ALM'
extracted_contents = []
for file_name in entries:
    assert file_name[0:12] == 'neural_data_'
    assert file_name[-(5+len(brain_region)):] == f'_{brain_region}.mat'
    extracted_contents.append(int(file_name[12:-(5+len(brain_region))]))


In [17]:
mapping_brain_to_days = {}
mapping_days_to_brain = {}

for brain_region in brain_region_list:
    dir_path = f"../data/raw_data/neural_data_{brain_region}"
    entries = os.listdir(dir_path)
    extracted_contents = []
    for file_name in entries:
        # Verify file naming convention; adjust the indices as necessary.
        assert file_name.startswith('neural_data_')
        assert file_name.endswith(f'_{brain_region}.mat')
        # Remove 'neural_data_' from the start and f'_{brain_region}.mat' from the end.
        num = int(file_name[len('neural_data_') : - (len(brain_region) + 5)])
        extracted_contents.append(num)
    extracted_contents.sort()
        
    mapping_brain_to_days[brain_region] = extracted_contents
    for day in extracted_contents:
        if day not in mapping_days_to_brain.keys():
            mapping_days_to_brain[day] = [brain_region]
        else:
            mapping_days_to_brain[day].append(brain_region)

print(mapping_brain_to_days)
print(mapping_days_to_brain)

{'left_ALM': [0, 1, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40, 41, 42, 43, 44, 47, 48, 49, 50, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 68, 69, 72, 73, 76, 78, 79, 80, 82, 83, 84, 89, 90, 91, 92, 93, 101, 102, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 125, 126, 127, 128, 129, 130, 131, 137, 138, 139, 140, 151, 152, 153, 154, 155, 156, 157, 160, 161, 162, 163, 164, 167, 168, 169], 'left_BLA': [86, 97, 101, 102, 116, 130, 140, 153, 154], 'left_ECT': [85, 96, 98, 99, 100, 113, 114, 115, 131, 132, 137, 138, 139, 151, 152, 155, 156, 157], 'left_Medulla': [57, 58, 59, 60, 61, 62, 63, 66, 67, 70, 71, 74, 75, 76, 77, 78, 79, 165, 166, 170, 171, 172], 'left_Midbrain': [13, 14, 15, 16, 21, 22, 23, 24, 25, 31, 32, 33, 34, 41, 42, 43, 44, 47, 48, 49, 50, 64, 65, 68, 69, 72, 73, 80, 81, 85, 87, 88, 92, 93, 94, 95, 96, 97, 108, 109, 112, 114, 115, 116, 121, 122, 123, 124, 131, 132, 137, 138, 139, 140, 151, 15